# LSTM Model


## Import Data
### Players from RPS_game.py

In [1]:
from Lib.RPS_game import play, mrugesh, abbey, quincy, kris, human, random_player
from Lib.RPS_encoding import RPS_encode, RPS_decode, RPS_encode_dataframe
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, LSTM, Embedding
import pandas as pd
from os import listdir
from os.path import isfile, join

## Read and Concatenate Games
Take all games and load into dataframe

In [3]:
path = "Games/"
all_game_paths = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
games_concat = pd.DataFrame(columns = ["Player","Opponent","Result","Winning_Play"])
games = []
for game_path in all_game_paths:
    #print(game_path)
    game_data = pd.read_csv(game_path, names = ["Player","Opponent","Result","Winning_Play"])
    games.append(game_data)
    games_concat = games_concat.append(game_data)
    
#print(len(game_data.index)) 
#game_data.head()  
print(len(games_concat.index))
games_concat.head()

80000


,Player,Opponent,Result,Winning_Play
0,R,R,0,P
1,S,P,1,S
2,S,P,1,S
3,P,P,0,S
4,R,P,0,S


# LSTM Training
### Parameters

In [4]:
seq_length = 100 #how far back the ltsm looks when training
#Model parameters
BATCH_SIZE = 64
VOCAB_SIZE = 3 # RPS
EMBEDDING_DIM = 256
RNN_UNITS = 1024

TRAINING_EPOCHS = 100

#best so far
#100
#64
#3
#128
#512
#200


### Format data for LTSM training

In [6]:
temp = games_concat.drop("Result", axis = 1)
temp = RPS_encode_dataframe(temp)
input_data = tf.data.Dataset.from_tensor_slices(temp["Opponent"])
target_data = tf.data.Dataset.from_tensor_slices(temp["Winning_Play"])
print(temp)
print(input_data)

#split into sequences
input_sequences = input_data.batch(seq_length+1, drop_remainder = True)
target_sequences = target_data.batch(seq_length+1, drop_remainder = True)
print(input_sequences)

#remove last element from all input sequences
def remove_last(chunk):
    return chunk[:-1]
inputs = input_sequences.map(remove_last)

#Code for abbey training
#inputs = target_sequences.map(remove_last)

#remove first element from all target sequences
def remove_first(chunk):
    return chunk[1:]
targets = target_sequences.map(remove_first)

dataset = tf.data.Dataset.zip((inputs, targets))
for element in dataset:
    print(element)
    break


     Player  Opponent  Winning_Play
0         0         0             1
1         2         1             2
2         2         1             2
3         1         1             2
4         0         1             2
..      ...       ...           ...
995       1         0             1
996       2         1             2
997       2         1             2
998       0         2             0
999       1         0             1

[80000 rows x 3 columns]
<TensorSliceDataset shapes: (), types: tf.int64>
<BatchDataset shapes: (101,), types: tf.int64>
(<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 2, 2, 1, 1,
       1, 2, 1, 0, 2, 2, 1, 0, 2, 0, 1, 1, 0, 2, 1, 2, 2, 1, 0, 2, 0, 1,
       2, 0, 1, 0, 2, 0, 0, 0, 1, 0, 1, 1, 1, 1, 2, 1, 1, 0, 0, 0, 1, 1,
       2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1,
       1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2], dtype=int64)>, <tf.Tensor: shape=(100,), dtype=int64, numpy=
ar

### Build the model

In [26]:
#Shuffle sequences before batching
data = dataset.shuffle(10000).batch(BATCH_SIZE, drop_remainder = True)

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.Sequential()
    model.add(Embedding(vocab_size, embedding_dim, batch_input_shape = [batch_size, None]))
    model.add(LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size+1))
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           768       
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_2 (Dense)              (64, None, 4)             4100      
Total params: 5,251,844
Trainable params: 5,251,844
Non-trainable params: 0
_________________________________________________________________


In [27]:
#RUN TO CONTINUE TRAINING FROM LAST WEIGHTS
model.load_weights('Models/tmpWeights')

### Create loss function

In [28]:
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits = True)

### Compile the model

In [29]:
model.compile(optimizer='adam', loss=loss)

### Create Checkpoints
configure the model to create checkpoints as it trains so the model can be reloaded and continue training

In [30]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath = checkpoint_prefix, save_weights_only = True)

print(tf.__version__)

2.3.1


### Training the model

In [31]:
#Program can crash when training due to tensorflow 2.0 GPU issue
history = model.fit(data, epochs = TRAINING_EPOCHS, callbacks = [checkpoint_callback])

Epoch 1/100
12/12 [==============================] - 0s 40ms/step - loss: 0.1166
Epoch 2/100
12/12 [==============================] - 0s 37ms/step - loss: 0.1182
Epoch 3/100
12/12 [==============================] - 0s 37ms/step - loss: 0.1113
Epoch 4/100
12/12 [==============================] - 0s 38ms/step - loss: 0.1042
Epoch 5/100
12/12 [==============================] - 0s 38ms/step - loss: 0.1087: 0s - loss: 0
Epoch 6/100
12/12 [==============================] - 0s 37ms/step - loss: 0.1135
Epoch 7/100
12/12 [==============================] - 0s 37ms/step - loss: 0.1075
Epoch 8/100
12/12 [==============================] - 0s 38ms/step - loss: 0.1001
Epoch 9/100
12/12 [==============================] - 0s 38ms/step - loss: 0.1053
Epoch 10/100
12/12 [==============================] - 0s 37ms/step - loss: 0.1042
Epoch 11/100
12/12 [==============================] - 0s 37ms/step - loss: 0.0998
Epoch 12/100
12/12 [==============================] - 1s 42ms/step - loss: 0.1013
Epoch 13/10

12/12 [==============================] - 1s 42ms/step - loss: 0.0854


In [32]:
model.save_weights("Models/tmpWeights")

### Rebuild model for predicting one output rather than a vector

In [33]:
#change config
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size = 1)
#load trained weights from checkpoint
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

## Generate Outputs from Model

In [20]:
def Get_Prediction(model, opponent_history):
    #format history for input
    input_eval = [0 if c == '' else RPS_encode(c) for c in opponent_history]
    input_eval = tf.expand_dims(input_eval, 0)
    
    #high temperature  => suprising output
    #low temperature => predictable output
    temperature = 1; 
    
    model.reset_states()
    
    prediction = model(input_eval)
    prediction = tf.squeeze(prediction, 0)
    
    prediction = prediction / temperature
    predicted_id = tf.random.categorical(prediction, num_samples = 1)[-1,0].numpy()
    
    return RPS_decode(predicted_id)

#print(Get_Prediction(model, ['R','R','P']))

# RPS.py
Player function

In [21]:
plays = 1000

def player(prev_play, opponent_history=[]):
    if len(opponent_history) >= plays: 
        opponent_history.clear()
        
    opponent_history.append(prev_play)

    guess = Get_Prediction(model, opponent_history)
    
    return guess

# MAIN.PY

In [34]:
play(player, quincy, plays)
play(player, abbey, plays)
play(player, kris, plays)
play(player, mrugesh, plays)

Final results: {'p1': 998, 'p2': 1, 'tie': 1}
Player 1 win rate: 99.8998998998999%
Final results: {'p1': 331, 'p2': 342, 'tie': 327}
Player 1 win rate: 49.18276374442793%
Final results: {'p1': 468, 'p2': 254, 'tie': 278}
Player 1 win rate: 64.81994459833795%
Final results: {'p1': 697, 'p2': 200, 'tie': 103}
Player 1 win rate: 77.70345596432553%


77.70345596432553

## Saving the model

In [35]:
model.save('Models/LSTM_BalancedTraining_OpponentBased_Quincy_Mrugesh')

INFO:tensorflow:Assets written to: Models/LSTM_BalancedTraining_OpponentBased_Quincy_Mrugesh\assets
